In [29]:
import pandas as pd
import pyodbc
from datetime import datetime

In [45]:
sql_query = """
    -- all renewal WO and their flocid
    SELECT
        [WorkOrderNumber],
        [OrderType],
        [CompanyCode],
        [FunctionLocation],
        CASE
            WHEN [TechCompletionDate] IS NULL THEN [BasicFinishDate]
            ELSE [TechCompletionDate]
        END AS 'CompletionDate',
        [MaintenanceActivityType],
        [MaintenanceActivityTypeDesc],
        [MainUserStatus],
        [MainUserStatusDesc],
        [ActualTotalCost]
    FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.ringfenced].[vw_WorkOrder]
    WHERE
        OrderType = 'MW04'
        AND CompanyCode = '5000'
        AND MainUserStatusDesc = 'Practically Completed'
        AND ActualTotalCost IS NOT NULL
        AND ActualTotalCost > 0
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';DATABASE=myANALYTICS_SP;Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_renewal_wo = pd.read_sql_query(sql_query, conn)

all_renewal_wo.to_pickle("../pkl/work_order_all.pkl")
# Display the DataFrame
all_renewal_wo['CompletionDate'] = pd.to_datetime(all_renewal_wo['CompletionDate'])

all_renewal_wo

C:\Users\R893859\AppData\Local\Temp\ipykernel_20220\2620353103.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_renewal_wo = pd.read_sql_query(sql_query, conn)


,WorkOrderNumber,OrderType,CompanyCode,FunctionLocation,CompletionDate,MaintenanceActivityType,MaintenanceActivityTypeDesc,MainUserStatus,MainUserStatusDesc,ActualTotalCost
0,000080115566,MW04,5000,GA-02BA,2024-05-03,C14,Ballast Excavator U/C Major,PCOM,Practically Completed,14447.47
1,000080077696,MW04,5000,NL-02ML,2017-06-02,C05,Formation Renewal,PCOM,Practically Completed,282889.66
2,000080077698,MW04,5000,BW-08ML,2017-09-08,C05,Formation Renewal,PCOM,Practically Completed,323047.55
3,000080077699,MW04,5000,GA-02ML,2017-04-27,C17,Rail Renewal,PCOM,Practically Completed,475.00
4,000080077821,MW04,5000,TO000617,2017-03-23,C16,Turnout Renew Maj Civil Parts,PCOM,Practically Completed,55357.97
...,...,...,...,...,...,...,...,...,...,...
9924,000080117705,MW04,5000,None,2024-06-04,C40,Level Xing Design,PCOM,Practically Completed,3024.45
9925,000080117713,MW04,5000,CV006262,2024-06-20,C21,Track Upgrade,PCOM,Practically Completed,11302.80
9926,000080117722,MW04,5000,BW-01ML,2024-06-16,C05,Formation Renewal,PCOM,Practically Completed,543.08
9927,000080117727,MW04,5000,TO000845,2024-06-27,C20,Bridge Ballast Renewal,PCOM,Practically Completed,10955.02


In [46]:
sql_query = """
    SELECT DISTINCT
        FLOC_STRNO_FunctionalLocation,
        IFLOT_DATAB_StartupDate
    FROM
    myANALYTICS_SP.[workarea.silver.dimension.enterprise.asset].vw_Dim_FunctionalLocationStartupDate
    WHERE IFLOT_DATAB_StartupDate > 0
    AND FLOC_STRNO_FunctionalLocation IS NOT NULL
    AND FLOC_BUKRS_CompanyCode = '5000'
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';DATABASE=myANALYTICS_SP;Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
startupdate_df = pd.read_sql_query(sql_query, conn)

# df1.to_pickle("../pkl/work_order_all.pkl")
# Display the DataFrame
startupdate_df['IFLOT_DATAB_StartupDate'] = pd.to_datetime(startupdate_df['IFLOT_DATAB_StartupDate'])
startupdate_df

C:\Users\R893859\AppData\Local\Temp\ipykernel_20220\2242733206.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  startupdate_df = pd.read_sql_query(sql_query, conn)


,FLOC_STRNO_FunctionalLocation,IFLOT_DATAB_StartupDate
0,ST000480-63,2014-01-01
1,ST000850-48,1994-01-01
2,ST000960-87,1987-01-01
3,ST008220-29,2011-11-16
4,ST003870-03,2013-01-01
...,...,...
25962,ST004120-17,2016-05-29
25963,ST006730-01,2007-12-31
25964,ST004700-08,2004-12-31
25965,TO004059,2021-01-01


In [47]:
merged_df1 = pd.merge(startupdate_df, all_renewal_wo, how='outer', left_on='FLOC_STRNO_FunctionalLocation', right_on='FunctionLocation')
merged_df1 = merged_df1[['FLOC_STRNO_FunctionalLocation', 'IFLOT_DATAB_StartupDate', 'CompletionDate']]

# merged_df1[merged_df1['IFLOT_DATAB_StartupDate'].notna() & merged_df1['CompletionDate'].notna()]
# merged_df1[merged_df1['FLOC_STRNO_FunctionalLocation'].notna() & merged_df1['CompletionDate'].notna()]
merged_df1

,FLOC_STRNO_FunctionalLocation,IFLOT_DATAB_StartupDate,CompletionDate
0,AR-01CR-CHCH,2021-01-01,NaT
1,AR-01CR-CHCH-ST0010,2021-01-01,NaT
2,AR-01ML,2021-01-01,NaT
3,AR-01ML-JBJB,2021-01-01,NaT
4,AR-01ML-JBJB-TO0010,2021-01-01,NaT
...,...,...,...
35647,NaN,NaT,2024-05-29
35648,NaN,NaT,2024-07-04
35649,NaN,NaT,2024-06-04
35650,NaN,NaT,2024-06-04


In [48]:
# Getting all FLOC from [vw_Dim_FunctionalLocation] where CompanyCode = 5000 -> df2_no_length
sql_query = """
	-- Converting Civil Structures Asset Renewal from Python to SQL
	WITH DF1 AS(
		SELECT [FuncLocID]
			,[STRUCTURE_LENGTH]
			,[YEAR_OF_CONSTRUCTION]
			,[LinearRefPattern]
			,(CASE
					WHEN [ASSET_CLASS_TYPE] = 'BRIDGE' AND [SPAN_MATERIAL] = 'STEEL' THEN 'Steel Bridge'
					WHEN [ASSET_CLASS_TYPE] = 'BRIDGE' AND [SPAN_MATERIAL] = 'TIMBER' THEN 'Timber Bridge'
					ELSE NULL
				END) AS [Asset_Type_Bridge]
			,(CASE
					WHEN [STRUCTURE_TYPE] = 'REINFORCED CONCRETE BOXCULVERT' AND CAST(SUBSTRING(CAST([YEAR_OF_CONSTRUCTION] AS VARCHAR), 1, 4) AS FLOAT) < 2000 THEN 'Reinforced Concrete Box Culvert (RCBC) prior to year 2000'
					WHEN [STRUCTURE_TYPE] = 'REINFORCED CONCRETE BOXCULVERT' AND CAST(SUBSTRING(CAST([YEAR_OF_CONSTRUCTION] AS VARCHAR), 1, 4) AS FLOAT) >= 2000 THEN 'Reinforced Concrete Box Culvert (RCBC) designed to AS1597'
					ELSE NULL
				END) AS Asset_Type_Boxculvert
			,(CASE 
					WHEN [STRUCTURE_TYPE] = 'REINFORCED PIPE CULVERT' THEN 'Reinforced Concrete Pipe (RCP)'
					WHEN [STRUCTURE_TYPE] = 'CORRUGATED METAL PIPE' THEN 'Corrugated Metal Pipe (CMP)'
					WHEN [STRUCTURE_TYPE] != 'REINFORCED PIPE CULVERT' AND [SPAN_TYPE] = 'PIPE CULVERT' AND SPAN_MATERIAL LIKE '%CONCRETE%' THEN 'Concrete Pipe (Unreinforced)'
					ELSE NULL
				END) AS [Asset_Type_Pipe]
			,(CASE 
					WHEN [SPAN_MATERIAL] = 'CAST-IN-SITU CONCRETE' THEN 'Concrete In situ Arch'
					ELSE NULL
				END) AS [Asset_Type_Arch]
			,(CASE 
					WHEN [FuncLocID] LIKE 'RW%' THEN 'Concrete Retaining Wall'
					ELSE NULL
				END) AS [Asset_Type_Wall]
			,(CASE 
					WHEN SPAN_MATERIAL = 'STEEL' AND STRUCTURE_USAGE = 'PEDESTRIAN' AND TechnicalObjectTypeDesc = 'Bridges' THEN 'Pedestrian Overbridge (Steel)'
					ELSE NULL
				END) AS [Asset_Type_Overbridge]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.civil.structure.ringfenced].[vw_FLOCStructure]
		WHERE 1=1
		AND [UserStatusDesc]  = 'In Service'
	),

	DF2 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[STRUCTURE_LENGTH]
			,[YEAR_OF_CONSTRUCTION]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[STRUCTURE_LENGTH]
				,[YEAR_OF_CONSTRUCTION]
				,[LinearRefPattern]
				,CAST([Asset_Type_Bridge] AS VARCHAR(100)) AS [Asset_Type_Bridge]
				,CAST([Asset_Type_Boxculvert] AS VARCHAR(100)) AS [Asset_Type_Boxculvert]
				,CAST([Asset_Type_Pipe] AS VARCHAR(100)) AS [Asset_Type_Pipe]
				,CAST([Asset_Type_Arch] AS VARCHAR(100)) AS [Asset_Type_Arch]
				,CAST([Asset_Type_Wall] AS VARCHAR(100)) AS [Asset_Type_Wall]
				,CAST([Asset_Type_Overbridge] AS VARCHAR(100)) AS [Asset_Type_Overbridge]
			FROM DF1) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([Asset_Type_Bridge]
				,[Asset_Type_Boxculvert]
				,[Asset_Type_Pipe]
				,[Asset_Type_Arch]
				,[Asset_Type_Wall]
				,[Asset_Type_Overbridge])
			) AS unpvt
	)

	SELECT * FROM DF2
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_floc = pd.read_sql_query(sql_query, conn)

# Display the DataFrame
all_floc

C:\Users\R893859\AppData\Local\Temp\ipykernel_20220\2758387442.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_floc = pd.read_sql_query(sql_query, conn)


,FuncLocID,ASSET,ASSET_TYPE,STRUCTURE_LENGTH,YEAR_OF_CONSTRUCTION,LinearRefPattern
0,BR400125,Steel Bridge,Asset_Type_Bridge,25,19680101,MA-01ML
1,BR400125,Pedestrian Overbridge (Steel),Asset_Type_Overbridge,25,19680101,MA-01ML
2,CU100001,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,0,19750101,BW-23ML
3,CU100004,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,0,19750101,BW-23ML
4,CU100009,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,0,19750101,BW-23ML
...,...,...,...,...,...,...
3849,CU400610,Concrete Pipe (Unreinforced),Asset_Type_Pipe,0,None,MA-03FK
3850,CU400615,Corrugated Metal Pipe (CMP),Asset_Type_Pipe,18,None,MA-01ML
3851,CU400616,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,19.52,20240101,MA-01ML
3852,RW000004,Concrete Retaining Wall,Asset_Type_Wall,None,19000101,BW-12ML


In [52]:
merged_df2 = pd.merge(all_floc, merged_df1, how='left', left_on='FuncLocID', right_on='FLOC_STRNO_FunctionalLocation')
merged_df2 = merged_df2[['FuncLocID', 'ASSET', 'YEAR_OF_CONSTRUCTION', 'IFLOT_DATAB_StartupDate', 'CompletionDate']]
merged_df2['YEAR_OF_CONSTRUCTION'] = pd.to_datetime(merged_df2['YEAR_OF_CONSTRUCTION'], format='%Y%m%d', errors='coerce')

# merged_df4 = merged_df4[merged_df4['IFLOT_DATAB_StartupDate'].notnull()]
merged_df2

,FuncLocID,ASSET,YEAR_OF_CONSTRUCTION,IFLOT_DATAB_StartupDate,CompletionDate
0,BR400125,Steel Bridge,1968-01-01,NaT,NaT
1,BR400125,Pedestrian Overbridge (Steel),1968-01-01,NaT,NaT
2,CU100001,Reinforced Concrete Pipe (RCP),1975-01-01,NaT,NaT
3,CU100004,Reinforced Concrete Pipe (RCP),1975-01-01,NaT,NaT
4,CU100009,Reinforced Concrete Pipe (RCP),1975-01-01,NaT,NaT
...,...,...,...,...,...
3849,CU400610,Concrete Pipe (Unreinforced),NaT,NaT,NaT
3850,CU400615,Corrugated Metal Pipe (CMP),NaT,NaT,NaT
3851,CU400616,Reinforced Concrete Pipe (RCP),2024-01-01,NaT,NaT
3852,RW000004,Concrete Retaining Wall,1900-01-01,NaT,NaT


In [57]:
priority_order = ['YEAR_OF_CONSTRUCTION', 'IFLOT_DATAB_StartupDate', 'CompletionDate']
merged_df2['floc_age'] = merged_df2[priority_order].fillna(method='bfill', axis=1).iloc[:, 0]
merged_df2

C:\Users\R893859\AppData\Local\Temp\ipykernel_20220\632807999.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df2['floc_age'] = merged_df2[priority_order].fillna(method='bfill', axis=1).iloc[:, 0]


,FuncLocID,ASSET,YEAR_OF_CONSTRUCTION,IFLOT_DATAB_StartupDate,CompletionDate,floc_age
0,BR400125,Steel Bridge,1968-01-01,NaT,NaT,1968-01-01
1,BR400125,Pedestrian Overbridge (Steel),1968-01-01,NaT,NaT,1968-01-01
2,CU100001,Reinforced Concrete Pipe (RCP),1975-01-01,NaT,NaT,1975-01-01
3,CU100004,Reinforced Concrete Pipe (RCP),1975-01-01,NaT,NaT,1975-01-01
4,CU100009,Reinforced Concrete Pipe (RCP),1975-01-01,NaT,NaT,1975-01-01
...,...,...,...,...,...,...
3849,CU400610,Concrete Pipe (Unreinforced),NaT,NaT,NaT,NaT
3850,CU400615,Corrugated Metal Pipe (CMP),NaT,NaT,NaT,NaT
3851,CU400616,Reinforced Concrete Pipe (RCP),2024-01-01,NaT,NaT,2024-01-01
3852,RW000004,Concrete Retaining Wall,1900-01-01,NaT,NaT,1900-01-01


In [54]:
merged_df2[merged_df2['floc_age'].notna()]

,FuncLocID,ASSET,YEAR_OF_CONSTRUCTION,IFLOT_DATAB_StartupDate,CompletionDate,floc_age
0,BR400125,Steel Bridge,1968-01-01,NaT,NaT,1968-01-01
1,BR400125,Pedestrian Overbridge (Steel),1968-01-01,NaT,NaT,1968-01-01
2,CU100001,Reinforced Concrete Pipe (RCP),1975-01-01,NaT,NaT,1975-01-01
3,CU100004,Reinforced Concrete Pipe (RCP),1975-01-01,NaT,NaT,1975-01-01
4,CU100009,Reinforced Concrete Pipe (RCP),1975-01-01,NaT,NaT,1975-01-01
...,...,...,...,...,...,...
3847,CU400577,Concrete In situ Arch,2013-01-01,NaT,NaT,2013-01-01
3848,CU400605,Reinforced Concrete Pipe (RCP),1900-01-01,NaT,NaT,1900-01-01
3851,CU400616,Reinforced Concrete Pipe (RCP),2024-01-01,NaT,NaT,2024-01-01
3852,RW000004,Concrete Retaining Wall,1900-01-01,NaT,NaT,1900-01-01


In [137]:
# t_df1 = df1[['FunctionLocation', 'CompletionDate']].rename(columns = {'CompletionDate':'Date'}).copy()
# t_df1['Date'] = pd.to_datetime(t_df1['Date'])
# startupdate_df.rename(columns={'FLOC_STRNO_FunctionalLocation':'FunctionLocation','IFLOT_DATAB_StartupDate':'Date'},inplace=True)
# startupdate_df['Date'] = pd.to_datetime(startupdate_df['Date'])
# t = pd.concat([startupdate_df,t_df1],axis=0,ignore_index=True)
# t

In [138]:
# t.duplicated().sum()
# t['FunctionLocation'].nunique()
# t_df1['FunctionLocation'].nunique(), t_df1.shape

In [139]:
df2 = pd.read_pickle(r'C:\Users\R893859\Aurizon Operations Limited\Data, Reporting & Analytics - DS08 - NSAP2\Data\Huy\Flocs List\renewal_flocs.pkl')
df2

,FuncLocID,Asset Type,Category
0,SG000010,SIGNAL GANTRY,Control & Electrical
1,SG000012,SIGNAL GANTRY,Control & Electrical
2,SG000013,SIGNAL GANTRY,Control & Electrical
3,SG000015,SIGNAL GANTRY,Control & Electrical
4,SG000018,SIGNAL GANTRY,Control & Electrical
...,...,...,...
46321,ST001900-07,MOTORISED ISOLATOR MOTOR UNITS,Control & Electrical
46322,SQ030456,MOTORISED ISOLATOR MOTOR UNITS,Control & Electrical
46323,SQ030458,MOTORISED ISOLATOR MOTOR UNITS,Control & Electrical
46324,SQ030464,MOTORISED ISOLATOR MOTOR UNITS,Control & Electrical


In [140]:
all_asset_type_df = pd.DataFrame(df2['Asset Type'].unique(), columns=['Asset Type'])
all_asset_type_df

,Asset Type
0,SIGNAL GANTRY
1,LEVEL CROSSING MONITORS
2,TRAIN PROTECTION SYSTEMS (ATP)
3,WEATHER STATION
4,RADIO POLES
...,...
87,AUTOTRANSFORMERS
88,HARMONIC FILTERS
89,TRACTION SCADA
90,AUXILIARY SUPPLY TRANSFORMERS


In [141]:
merged_df = pd.merge(df1, df2, how='left', left_on='FunctionLocation', right_on='FuncLocID')

merged_df

,WorkOrderNumber,OrderType,CompanyCode,FunctionLocation,CompletionDate,MaintenanceActivityType,MaintenanceActivityTypeDesc,MainUserStatus,MainUserStatusDesc,ActualTotalCost,FuncLocID,Asset Type,Category
0,000080079052,MW04,5000,MA-01ML,2017-09-14,C14,Ballast Excavator U/C Major,PCOM,Practically Completed,17710.00,NaN,NaN,NaN
1,000080079053,MW04,5000,TO000943,2017-07-14,C16,Turnout Renew Maj Civil Parts,PCOM,Practically Completed,2760.00,TO000943,1 IN 12 (60KG/M) RBM POINTS AND CROSSING,Track & Ballast
2,000080079059,MW04,5000,BW-04ML,2017-09-08,C17,Rail Renewal,PCOM,Practically Completed,27073.35,NaN,NaN,NaN
3,000080079060,MW04,5000,TO000486,2018-04-04,C63,Joint Renewal,PCOM,Practically Completed,10374.13,TO000486,1 IN 16 (60KG/M) SWING NOSE POINTS AND CROSSING,Track & Ballast
4,000080079158,MW04,5000,BW-04ML,2017-09-14,C13,Ballast Undercut Full Turnout,PCOM,Practically Completed,38073.08,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10217,000080116924,MW04,5000,CU101375,2024-05-02,B52,Culvert Renewal,PCOM,Practically Completed,17443.07,CU101375,REINFORCED CONCRETE BOX CULVERT (RCBC) PRIOR T...,Structure & Civils
10218,000080117107,MW04,5000,None,2024-04-17,B56,Ancillary structure const.,PCOM,Practically Completed,1900.69,NaN,NaN,NaN
10219,000080117713,MW04,5000,CV006262,2024-06-05,C21,Track Upgrade,PCOM,Practically Completed,3809.52,CV006262,60KG/M RAIL - HH CURVES (601 TO 1000 METRE RAD...,Track & Ballast
10220,000080117713,MW04,5000,CV006262,2024-06-05,C21,Track Upgrade,PCOM,Practically Completed,3809.52,CV006262,28.0 TONNE AXLE LOAD PSC SLEEPERS,Track & Ballast


In [142]:
# nan_count = merged_df['Asset Type'].isna().sum()
# nan_count

7718

In [143]:
# Convert CompletionDate column into datetime
merged_df['CompletionDate'] = pd.to_datetime(merged_df['CompletionDate'])
# merged_df['Asset Type'] = merged_df['Asset Type'].fillna('Unknown')

# Calculate all time average
grouped_all_time = merged_df.groupby('Asset Type').agg(
    Average_ActualTotalCost_All_Time=('ActualTotalCost', 'mean'),
    Sample_Size_All_Time=('Asset Type', 'size')
).reset_index()

grouped_all_time

,Asset Type,Average_ActualTotalCost_All_Time,Sample_Size_All_Time
0,1 IN 12 (47KG/M) FABRICATED POINTS AND CROSSING,53840.865385,13
1,1 IN 12 (53KG/M) FABRICATED POINTS AND CROSSING,43754.905455,11
2,1 IN 12 (53KG/M) RBM POINTS AND CROSSING,25101.284000,20
3,1 IN 12 (60KG/M) RBM POINTS AND CROSSING,40457.647655,145
4,1 IN 12 (60KG/M) SPRING WING CROSSING,41477.865843,89
...,...,...,...
41,REPLACE PANDROL E-CLIP FASTENINGS (<10KM FROM ...,176609.025746,181
42,SECTION INSULATORS,35706.427692,26
43,TRACTION POWER SUPPLY TRANSFORMERS,83722.333333,15
44,UPS,41392.480000,2


In [144]:
# Calculate last year average
current_year = datetime.now().year
previous_year = current_year - 1

df_previous_year = merged_df[merged_df['CompletionDate'].dt.year == previous_year]

grouped_previous_year = df_previous_year.groupby('Asset Type').agg(
    Average_ActualTotalCost_Previous_Year=('ActualTotalCost', 'mean'),
    Sample_Size_Previous_Year=('Asset Type', 'size')
).reset_index()

grouped_previous_year

,Asset Type,Average_ActualTotalCost_Previous_Year,Sample_Size_Previous_Year
0,1 IN 12 (47KG/M) FABRICATED POINTS AND CROSSING,125902.406000,5
1,1 IN 12 (53KG/M) RBM POINTS AND CROSSING,8230.625000,2
2,1 IN 12 (60KG/M) RBM POINTS AND CROSSING,40136.250000,33
3,1 IN 12 (60KG/M) SPRING WING CROSSING,26773.940000,18
4,1 IN 12 (60KG/M) SWING NOSE POINTS AND CROSSING,47786.222909,55
...,...,...,...
31,REPLACE PANDROL E-CLIP FASTENINGS (<10KM FROM ...,184903.414086,93
32,SECTION INSULATORS,27413.282000,20
33,TRACTION POWER SUPPLY TRANSFORMERS,83633.158000,5
34,UPS,44.000000,1


In [145]:
# Calculate last 3 years average
last_3_years_start = current_year - 3

df_last_3_years = merged_df[(merged_df['CompletionDate'].dt.year >= last_3_years_start) & (merged_df['CompletionDate'].dt.year != current_year)]

grouped_last_3_years = df_last_3_years.groupby('Asset Type').agg(
    Average_Last_3_Years=('ActualTotalCost', 'mean'),
    Sample_Size_Last_3_Years=('Asset Type', 'size')
).reset_index()

grouped_last_3_years

,Asset Type,Average_Last_3_Years,Sample_Size_Last_3_Years
0,1 IN 12 (47KG/M) FABRICATED POINTS AND CROSSING,125902.406000,5
1,1 IN 12 (53KG/M) FABRICATED POINTS AND CROSSING,30655.857500,4
2,1 IN 12 (53KG/M) RBM POINTS AND CROSSING,9420.830000,6
3,1 IN 12 (60KG/M) RBM POINTS AND CROSSING,44632.890723,83
4,1 IN 12 (60KG/M) SPRING WING CROSSING,25667.624490,49
...,...,...,...
39,REPLACE PANDROL E-CLIP FASTENINGS (<10KM FROM ...,142057.057500,132
40,SECTION INSULATORS,37642.132727,22
41,TRACTION POWER SUPPLY TRANSFORMERS,68045.527000,10
42,UPS,41392.480000,2


In [146]:
# Calculate last 5 years average
last_5_years_start = current_year - 5

df_last_5_years = merged_df[(merged_df['CompletionDate'].dt.year >= last_5_years_start) & (merged_df['CompletionDate'].dt.year != current_year)]

grouped_last_5_years = df_last_5_years.groupby('Asset Type').agg(
    Average_Last_5_Years=('ActualTotalCost', 'mean'),
    Sample_Size_Last_5_Years=('Asset Type', 'size')
).reset_index()

grouped_last_5_years

,Asset Type,Average_Last_5_Years,Sample_Size_Last_5_Years
0,1 IN 12 (47KG/M) FABRICATED POINTS AND CROSSING,71096.215556,9
1,1 IN 12 (53KG/M) FABRICATED POINTS AND CROSSING,29344.892857,7
2,1 IN 12 (53KG/M) RBM POINTS AND CROSSING,21808.769167,12
3,1 IN 12 (60KG/M) RBM POINTS AND CROSSING,42726.383178,107
4,1 IN 12 (60KG/M) SPRING WING CROSSING,26741.093284,67
...,...,...,...
40,REPLACE PANDROL E-CLIP FASTENINGS (<10KM FROM ...,142057.057500,132
41,SECTION INSULATORS,37642.132727,22
42,TRACTION POWER SUPPLY TRANSFORMERS,68045.527000,10
43,UPS,41392.480000,2


In [147]:
result_df = pd.merge(grouped_all_time, grouped_previous_year, on='Asset Type', how='left')
result_df = pd.merge(result_df, grouped_last_3_years, on='Asset Type', how='left')
result_df = pd.merge(result_df, grouped_last_5_years, on='Asset Type', how='left')

result_df

,Asset Type,Average_ActualTotalCost_All_Time,Sample_Size_All_Time,Average_ActualTotalCost_Previous_Year,Sample_Size_Previous_Year,Average_Last_3_Years,Sample_Size_Last_3_Years,Average_Last_5_Years,Sample_Size_Last_5_Years
0,1 IN 12 (47KG/M) FABRICATED POINTS AND CROSSING,53840.865385,13,125902.406000,5.0,125902.406000,5.0,71096.215556,9.0
1,1 IN 12 (53KG/M) FABRICATED POINTS AND CROSSING,43754.905455,11,NaN,NaN,30655.857500,4.0,29344.892857,7.0
2,1 IN 12 (53KG/M) RBM POINTS AND CROSSING,25101.284000,20,8230.625000,2.0,9420.830000,6.0,21808.769167,12.0
3,1 IN 12 (60KG/M) RBM POINTS AND CROSSING,40457.647655,145,40136.250000,33.0,44632.890723,83.0,42726.383178,107.0
4,1 IN 12 (60KG/M) SPRING WING CROSSING,41477.865843,89,26773.940000,18.0,25667.624490,49.0,26741.093284,67.0
...,...,...,...,...,...,...,...,...,...
41,REPLACE PANDROL E-CLIP FASTENINGS (<10KM FROM ...,176609.025746,181,184903.414086,93.0,142057.057500,132.0,142057.057500,132.0
42,SECTION INSULATORS,35706.427692,26,27413.282000,20.0,37642.132727,22.0,37642.132727,22.0
43,TRACTION POWER SUPPLY TRANSFORMERS,83722.333333,15,83633.158000,5.0,68045.527000,10.0,68045.527000,10.0
44,UPS,41392.480000,2,44.000000,1.0,41392.480000,2.0,41392.480000,2.0


In [150]:
result_df = pd.merge(all_asset_type_df, result_df, on='Asset Type', how='left')
result_df.to_pickle("../pkl/average_renewal_cost.pkl")
result_df.to_excel('../csv/average_renewal_cost.xlsx', index=False)

result_df

,Asset Type,Average_ActualTotalCost_All_Time,Sample_Size_All_Time,Average_ActualTotalCost_Previous_Year,Sample_Size_Previous_Year,Average_Last_3_Years,Sample_Size_Last_3_Years,Average_Last_5_Years,Sample_Size_Last_5_Years
0,SIGNAL GANTRY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LEVEL CROSSING MONITORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN PROTECTION SYSTEMS (ATP),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WEATHER STATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RADIO POLES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
87,AUTOTRANSFORMERS,83722.333333,15.0,83633.158000,5.0,68045.527000,10.0,68045.527000,10.0
88,HARMONIC FILTERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,TRACTION SCADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,AUXILIARY SUPPLY TRANSFORMERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
all_category_type_df = pd.DataFrame(df2['Category'].unique(), columns=['Category'])

merged_df2 = pd.merge(df1, df2, how='left', left_on='FunctionLocation', right_on='FuncLocID')

# Convert CompletionDate column into datetime
merged_df2['CompletionDate'] = pd.to_datetime(merged_df2['CompletionDate'])

# Calculate all time average
grouped_all_time2 = merged_df2.groupby('Category').agg(
    Average_ActualTotalCost_All_Time=('ActualTotalCost', 'mean'),
    Sample_Size_All_Time=('Category', 'size')
).reset_index()

# Calculate last year average
current_year = datetime.now().year
previous_year = current_year - 1

df_previous_year2 = merged_df2[merged_df2['CompletionDate'].dt.year == previous_year]

grouped_previous_year2 = df_previous_year2.groupby('Category').agg(
    Average_ActualTotalCost_Previous_Year=('ActualTotalCost', 'mean'),
    Sample_Size_Previous_Year=('Category', 'size')
).reset_index()

# Calculate last 3 years average
last_3_years_start = current_year - 3

df_last_3_years2 = merged_df2[(merged_df2['CompletionDate'].dt.year >= last_3_years_start) & (merged_df2['CompletionDate'].dt.year != current_year)]

grouped_last_3_years2 = df_last_3_years2.groupby('Category').agg(
    Average_Last_3_Years=('ActualTotalCost', 'mean'),
    Sample_Size_Last_3_Years=('Category', 'size')
).reset_index()

# Calculate last 5 years average
last_5_years_start = current_year - 5

df_last_5_years2 = merged_df2[(merged_df2['CompletionDate'].dt.year >= last_5_years_start) & (merged_df2['CompletionDate'].dt.year != current_year)]

grouped_last_5_years2 = df_last_5_years2.groupby('Category').agg(
    Average_Last_5_Years=('ActualTotalCost', 'mean'),
    Sample_Size_Last_5_Years=('Category', 'size')
).reset_index()

# Join 3 df into 1
result_df2 = pd.merge(grouped_all_time2, grouped_previous_year2, on='Category', how='left')
result_df2 = pd.merge(result_df2, grouped_last_3_years2, on='Category', how='left')
result_df2 = pd.merge(result_df2, grouped_last_5_years2, on='Category', how='left')

result_df2 = pd.merge(all_category_type_df, result_df2, on='Category', how='left')
# result_df2.to_pickle("../pkl/average_renewal_cost.pkl")

result_df2

,Category,Average_ActualTotalCost_All_Time,Sample_Size_All_Time,Average_ActualTotalCost_Previous_Year,Sample_Size_Previous_Year,Average_Last_3_Years,Sample_Size_Last_3_Years,Average_Last_5_Years,Sample_Size_Last_5_Years
0,Control & Electrical,36221.359314,306,41017.736019,103,33382.607220,241,32578.600161,249
1,Track & Ballast,80866.122428,1948,111122.624670,621,73371.237494,1221,66776.755003,1475
2,Structure & Civils,23028.663120,250,10859.298833,120,12222.082593,189,14059.584948,194
